In [40]:
import pandas as pd
import numpy as np
from pathlib import Path
import xml.etree.ElementTree as ET

# WP: Wahlperiode
# Quelle der Daten: 

In [53]:
DATA_PATH = Path('data')
MDB_XML_PATH = DATA_PATH / 'MDB_STAMMDATEN.XML'
DF_MDB_PATH = DATA_PATH / 'df_mdb.csv' #one row per individuum, one col per wp containing 0 / 1
DF_MDB_WP_PATH = DATA_PATH / 'df_mdb_wp.csv' # one row per individuum per wp, containting e.g. 12. so one abgeordneter who has been in parliament for several wp will receive one row per wp

# parse xml data into dataFrame / .csv fomat

In [42]:
def get_mdb_data(mdb_xml_path = MDB_XML_PATH):
    list_of_mdb = []
    MDB_STAMMDATEN = ET.parse(mdb_xml_path)
    root = MDB_STAMMDATEN.getroot()
    
    for mdb in root.findall('MDB'):
        mdb_dict = {}
        ID, NAMEN, BIOGRAFISCHE_ANGABEN, WAHLPERIODEN = mdb[0], mdb[1], mdb[2], mdb[3]

        mdb_dict['ID'] = ID.text
        mdb_dict['NACHNAME'] = NAMEN.find('NAME').find('NACHNAME').text  
        mdb_dict['VORNAME'] = NAMEN.find('NAME').find('VORNAME').text  

        #print(mdb.tag, mdb.text, mdb.findall('NACHNAME'))
        for element in BIOGRAFISCHE_ANGABEN:
            mdb_dict[element.tag] = element.text

        mdb_dict['ANZ_WAHLPERIODEN'] = len(list(WAHLPERIODEN))

        for wahlperiode in WAHLPERIODEN:
            mdb_dict[int(wahlperiode.find('WP').text)] = 1

        list_of_mdb.append(mdb_dict)
    return pd.DataFrame(list_of_mdb)

In [352]:
def get_mdb_wp_data(mdb_xml_path = MDB_XML_PATH):
    list_of_mdb = []
    MDB_STAMMDATEN = ET.parse(mdb_xml_path)
    root = MDB_STAMMDATEN.getroot()
    
    for mdb in root.findall('MDB'):
        mdb_dict = {}
        ID, NAMEN, BIOGRAFISCHE_ANGABEN, WAHLPERIODEN = mdb[0], mdb[1], mdb[2], mdb[3]

        mdb_dict['ID'] = ID.text
        mdb_dict['NACHNAME'] = NAMEN.find('NAME').find('NACHNAME').text  
        mdb_dict['VORNAME'] = NAMEN.find('NAME').find('VORNAME').text  

        #print(mdb.tag, mdb.text, mdb.findall('NACHNAME'))
        for element in BIOGRAFISCHE_ANGABEN:
            mdb_dict[element.tag] = element.text

        mdb_dict['ANZ_WAHLPERIODEN'] = len(list(WAHLPERIODEN))

        for wahlperiode in WAHLPERIODEN:
            # one seperate entry per wp
            mdb_dict = mdb_dict.copy()
            mdb_dict['WP']=int(wahlperiode.find('WP').text)
            list_of_mdb.append(mdb_dict)
    return pd.DataFrame(list_of_mdb)

In [358]:
df_mdb = get_mdb_data(MDB_XML_PATH)
print(df_mdb.shape)
df_mdb.head()

(4089, 34)


,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
0,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11000002,Achenbach,Ernst,09.04.1909,Siegen,None,02.12.1991,männlich,"verheiratet, 3 Kinder",evangelisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11000003,Ackermann,Annemarie,26.05.1913,Parabutsch,Jugoslawien,18.02.1994,weiblich,"verheiratet, 5 Kinder",katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11000004,Ackermann,Else,06.11.1933,Berlin,None,14.09.2019,weiblich,ledig,evangelisch,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11000005,Adam,Ulrich,09.06.1950,"Teterow, Kr. Teterow, Bezirk Neubrandenburg",None,None,männlich,"verheiratet, 2 Kinder",evangelisch,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [359]:
df_mdb_wp = get_mdb_wp_data()
print(df_mdb_wp.shape)

(11627, 16)


In [360]:
df_mdb_wp[df_mdb_wp.NACHNAME == 'Merkel']

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,BERUF,PARTEI_KURZ,VITA_KURZ,VEROEFFENTLICHUNGSPFLICHTIGES,ANZ_WAHLPERIODEN,WP
4588,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,12
4589,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,13
4590,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,14
4591,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,15
4592,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,16
4593,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,17
4594,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,18
4595,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19
9794,11003591,Merkel,Petra,18.09.1947,Berlin,None,None,weiblich,"geschieden, 1 Kind",evangelisch,Kaufmännische Angestellte,SPD,None,None,3,15
9795,11003591,Merkel,Petra,18.09.1947,Berlin,None,None,weiblich,"geschieden, 1 Kind",evangelisch,Kaufmännische Angestellte,SPD,None,None,3,16


# clean data

In [361]:
# ok, quite expected. some cleaning necessarry: 'BÜNDNIS 90/DIE GRÜNEN', 'DIE GRÜNEN/BÜNDNIS 90', 'GRÜNE'
for grünen_alias in ['BÜNDNIS 90/DIE GRÜNEN', 'DIE GRÜNEN/BÜNDNIS 90', 'GRÜNE']:
    df_mdb.replace(grünen_alias, 'DIE GRÜNEN', inplace=True)
    df_mdb_wp.replace(grünen_alias, 'DIE GRÜNEN', inplace=True)

# save as csv

In [362]:
df_mdb.to_csv(DF_MDB_PATH, index=None)
df_mdb_wp.to_csv(DF_MDB_WP_PATH, index=None)

# read in data. exploratory data analysis

In [363]:
df_mdb = pd.read_csv(DF_MDB_PATH, index_col=None)

In [364]:
df_mdb.columns

Index(['ID', 'NACHNAME', 'VORNAME', 'GEBURTSDATUM', 'GEBURTSORT',
       'GEBURTSLAND', 'STERBEDATUM', 'GESCHLECHT', 'FAMILIENSTAND', 'RELIGION',
       'BERUF', 'PARTEI_KURZ', 'VITA_KURZ', 'VEROEFFENTLICHUNGSPFLICHTIGES',
       'ANZ_WAHLPERIODEN', '5', '6', '7', '8', '9', '10', '11', '3', '4', '2',
       '12', '13', '14', '15', '16', '1', '17', '18', '19'],
      dtype='object')

In [365]:
df_mdb.tail()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
4084,11004958,Nölke,Matthias,08.01.1980,Kassel,NaN,NaN,männlich,ledig,konfessionslos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4085,11004959,Martin,Dorothee,21.01.1978,NaN,NaN,NaN,weiblich,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4086,11004960,Wetzel,Wolfgang,11.05.1968,Schlema,NaN,NaN,männlich,"verheiratet, 2 Kinder",römisch-katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4087,11004961,Natterer,Christian,19.02.1981,Tettnang / Deutschland,NaN,NaN,männlich,"verheiratet, 1 Kind",römisch-katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4088,11004962,Dahmen,Janosch,06.09.1981,NaN,NaN,NaN,männlich,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [366]:
df_mdb_wp[['ID', 'PARTEI_KURZ']].groupby('PARTEI_KURZ').count().sort_values(by='ID', ascending=False).head(10)

,ID
PARTEI_KURZ,
SPD,4250
CDU,4150
FDP,1036
CSU,993
DIE GRÜNEN,536
DIE LINKE.,297
AfD,90
PDS,77
Plos,41


In [367]:
# wie - nur 2 Geschlechter? for the records: heute ist der 02.05.2021
df_mdb_wp[['ID', 'GESCHLECHT']].groupby('GESCHLECHT').count().sort_values(by='ID', ascending=False).head()

,ID
GESCHLECHT,
männlich,9382
weiblich,2245


In [60]:
# auch hier: such diversity. wow
df_mdb_wp[['ID', 'RELIGION']].groupby('RELIGION').count().sort_values(by='ID', ascending=False).head(n=16)

,ID
RELIGION,
evangelisch,3406
ohne Angaben,3275
katholisch,2985
römisch-katholisch,831
evangelisch-lutherisch,366
konfessionslos,234
evangelisch-reformiert,34
Atheistin,12
Islam,9


In [62]:
# jetzt wirds richtig vielseitig. so viele Busfahrer und Krankenschwestern!
df_mdb_wp[['ID', 'BERUF']].groupby('BERUF').count().sort_values(by='ID', ascending=False).head(16).head(16)

,ID
BERUF,
Rechtsanwalt,567
Geschäftsführer,223
Landwirt,212
Angestellter,190
Rechtsanwalt und Notar,126
Journalist,117
Dipl.-Ingenieur,109
Kaufmann,104
Jurist,102


In [63]:
# gehen sie weiter. hier gibt es kein Klischee zu erkennen!
df_mdb_wp[['ID', 'FAMILIENSTAND']].groupby('FAMILIENSTAND').count().sort_values(by='ID', ascending=False).head(16)

,ID
FAMILIENSTAND,
"verheiratet, 2 Kinder",2726
"verheiratet, 3 Kinder",1816
verheiratet,1720
"verheiratet, 1 Kind",1365
keine Angaben,1043
"verheiratet, 4 Kinder",891
ledig,424
"verheiratet, 5 Kinder",329
"verheiratet, 6 Kinder",178


In [328]:
# deleteme
start_date=16
end_date=18
#wps = range(start_date, end_date)
#selected_df = pd.concat([df_mdb[df_mdb[str(i)] == 1] for i in range(start_date,end_date+1)]).drop_duplicates()# deleteme
selected_df = df_mdb_wp[(df_mdb_wp['WP']>= start_date) & (df_mdb_wp['WP']<= end_date)].sort_values(by='VEROEFFENTLICHUNGSPFLICHTIGES')
selected_df.head()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,BERUF,PARTEI_KURZ,VITA_KURZ,VEROEFFENTLICHUNGSPFLICHTIGES,ANZ_WAHLPERIODEN,WP
11266,11004426,Träger,Carsten,25.10.1973,Fürth,None,None,männlich,"verheiratet, 2 Kinder",evangelisch-lutherisch,Koordinator Kommunikation,SPD,Abgeschlossenes Studium der Politikwissenschaf...,3 Funktionen in Unternehmen: \nDBU Naturerbe G...,2,18
11265,11004426,Träger,Carsten,25.10.1973,Fürth,None,None,männlich,"verheiratet, 2 Kinder",evangelisch-lutherisch,Koordinator Kommunikation,SPD,Abgeschlossenes Studium der Politikwissenschaf...,3 Funktionen in Unternehmen: \nDBU Naturerbe G...,2,18
8721,11002799,Siebert,Bernd,17.10.1949,Gudensberg,None,None,männlich,"verheiratet, 2 Kinder",evangelisch,selbstständiger Kaufmann,CDU,Volksschule in Gudensberg-Dorla. König-Heinric...,4 Funktionen in Körperschaften und Anstalten d...,7,16
8720,11002799,Siebert,Bernd,17.10.1949,Gudensberg,None,None,männlich,"verheiratet, 2 Kinder",evangelisch,selbstständiger Kaufmann,CDU,Volksschule in Gudensberg-Dorla. König-Heinric...,4 Funktionen in Körperschaften und Anstalten d...,7,16
8715,11002799,Siebert,Bernd,17.10.1949,Gudensberg,None,None,männlich,"verheiratet, 2 Kinder",evangelisch,selbstständiger Kaufmann,CDU,Volksschule in Gudensberg-Dorla. König-Heinric...,4 Funktionen in Körperschaften und Anstalten d...,7,16


In [309]:
# deleteme
# keep occurences
NUM_DIM_KEEP = 8
values_to_keep = selected_df[['ID', 'FAMILIENSTAND']].groupby('FAMILIENSTAND').count().sort_values(by='ID', ascending=False).index[NUM_DIM_KEEP:2*NUM_DIM_KEEP]
values_to_keep

Index(['geschieden, 2 Kinder', 'verheiratet, 5 Kinder', '3 Kinder', '1 Kind',
       'verwitwet, 2 Kinder', '4 Kinder', 'geschieden, 1 Kind',
       'verheiratet, 6 Kinder'],
      dtype='object', name='FAMILIENSTAND')

In [310]:
levels = [values_to_keep, range(start_date, end_date+1)]
levels

[Index(['geschieden, 2 Kinder', 'verheiratet, 5 Kinder', '3 Kinder', '1 Kind',
        'verwitwet, 2 Kinder', '4 Kinder', 'geschieden, 1 Kind',
        'verheiratet, 6 Kinder'],
       dtype='object', name='FAMILIENSTAND'),
 range(16, 19)]

In [311]:
new_index = pd.MultiIndex.from_product(levels, names=['FAMILIENSTAND', 'WP'])
new_index

MultiIndex([( 'geschieden, 2 Kinder', 16),
            ( 'geschieden, 2 Kinder', 17),
            ( 'geschieden, 2 Kinder', 18),
            ('verheiratet, 5 Kinder', 16),
            ('verheiratet, 5 Kinder', 17),
            ('verheiratet, 5 Kinder', 18),
            (             '3 Kinder', 16),
            (             '3 Kinder', 17),
            (             '3 Kinder', 18),
            (               '1 Kind', 16),
            (               '1 Kind', 17),
            (               '1 Kind', 18),
            (  'verwitwet, 2 Kinder', 16),
            (  'verwitwet, 2 Kinder', 17),
            (  'verwitwet, 2 Kinder', 18),
            (             '4 Kinder', 16),
            (             '4 Kinder', 17),
            (             '4 Kinder', 18),
            (   'geschieden, 1 Kind', 16),
            (   'geschieden, 1 Kind', 17),
            (   'geschieden, 1 Kind', 18),
            ('verheiratet, 6 Kinder', 16),
            ('verheiratet, 6 Kinder', 17),
           

In [323]:
# deleteme
grouped = selected_df[['ID', 'WP', 'FAMILIENSTAND']].groupby(['FAMILIENSTAND', 'WP']).count()
grouped.head(16)

ID
FAMILIENSTAND                       WP    
1 Kind                              16  11
                                    17   2
                                    18   6
2 Kinder                            16   9
                                    17  13
                                    18  16
3 Kinder                            16   7
                                    17  13
                                    18   1
4 Kinder                            17   6
                                    18   5
Lebensgemeinschaft                  18   7
eheähnl. Lebensgemeinschaft, 1 Kind 18   1
eingetragene Lebenspartnerschaft    17   3
geschieden                          17   3
                                    18   2

In [324]:
grouped = grouped.reindex(new_index, fill_value=0)
grouped

ID
FAMILIENSTAND         WP    
geschieden, 2 Kinder  16   6
                      17  15
                      18  14
verheiratet, 5 Kinder 16  10
                      17  11
                      18   8
3 Kinder              16   7
                      17  13
                      18   1
1 Kind                16  11
                      17   2
                      18   6
verwitwet, 2 Kinder   16   3
                      17  11
                      18   2
4 Kinder              16   0
                      17   6
                      18   5
geschieden, 1 Kind    16   2
                      17   6
                      18   2
verheiratet, 6 Kinder 16   1
                      17   1
                      18   6

In [325]:
grouped.reset_index(inplace=True)
grouped

,FAMILIENSTAND,WP,ID
0,"geschieden, 2 Kinder",16,6
1,"geschieden, 2 Kinder",17,15
2,"geschieden, 2 Kinder",18,14
3,"verheiratet, 5 Kinder",16,10
4,"verheiratet, 5 Kinder",17,11
5,"verheiratet, 5 Kinder",18,8
6,3 Kinder,16,7
7,3 Kinder,17,13
8,3 Kinder,18,1
9,1 Kind,16,11


In [326]:
# deleteme
wps = sorted(list(set(grouped.WP)))
familienstands = list(set(grouped.FAMILIENSTAND))
traces_values = []


In [327]:
for familienstand in familienstands:
    trace = grouped[grouped['FAMILIENSTAND'] == familienstand].sort_values(by='WP').ID.values
    traces_values.append(trace)
traces_values

[array([1, 1, 6]),
 array([ 3, 11,  2]),
 array([10, 11,  8]),
 array([2, 6, 2]),
 array([0, 6, 5]),
 array([ 7, 13,  1]),
 array([11,  2,  6]),
 array([ 6, 15, 14])]

# quality checks

In [340]:
df_mdb_wp[(df_mdb_wp.WP==19) & (df_mdb_wp.PARTEI_KURZ == 'CDU')][['ID', 'BERUF']].groupby('BERUF').count().sort_values(by='ID', ascending=False).head(16)

,ID
BERUF,
Rechtsanwalt,57
Landwirtschaftsmeister,15
"Bundestagspräsident, Bundesminister a. D., Jurist",13
Bürgermeister a. D.,10
Dipl.-Betriebswirt (FH),10
Parl. Staatssekretär,9
"Jurist, Vors. der CDU/CSU-Fraktion",8
"Pfarrer, Baufacharbeiter, Parl. Staatssekretär",8
"Bundeskanzlerin, Dipl.-Physikerin",8


In [351]:
df_mdb_wp[(df_mdb_wp.WP==19) & (df_mdb_wp.PARTEI_KURZ == 'CDU')&  (df_mdb_wp.BERUF.str.contains('Bundeskanzlerin')) ]#.drop_duplicates()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,BERUF,PARTEI_KURZ,VITA_KURZ,VEROEFFENTLICHUNGSPFLICHTIGES,ANZ_WAHLPERIODEN,WP
4588,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19
4589,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19
4590,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19
4591,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19
4592,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19
4593,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19
4594,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19
4595,11001478,Merkel,Angela,17.07.1954,Hamburg,None,None,weiblich,verheiratet,evangelisch,"Bundeskanzlerin, Dipl.-Physikerin",CDU,1973 Abitur in Templin. Physikstudium Univ. Le...,4 Funktionen in Körperschaften und Anstalten d...,8,19


In [349]:
df_mdb_wp.shape

(11627, 16)

In [350]:
df_mdb_wp.drop_duplicates().shape

(4089, 16)